# Environment Setting Up

In [1]:
import os
from dotenv import load_dotenv

# Loading environment variables from .env
load_dotenv()

# Changing directory to main directory for easy data access
working_directory = os.getenv("WORKING_DIRECTORY")
os.chdir(working_directory)

# Checking the change
%pwd

'/workspaces/Live-Air-Quality'

In [2]:
from pathlib import Path

# Checking the change
print("Git folder exists:", Path(".git").exists())

Git folder exists: True


# 4. SQL

In [3]:
import duckdb as ddb
import pandas as pd
from pathlib import Path

location = Path("research/sql/air_quality.db")
conn = ddb.connect(location)

conn

In [4]:
# Checking if the data has been loaded properly form the database
data = conn.query("""
    SELECT COUNT(*) AS cnt
    FROM raw.air_quality_data;
""").to_df()

data.head()

,cnt
0,2877


In [5]:
# Creating a presentation layer in the Database
# This will take the data from raw and give us views that can be used
conn.execute("CREATE SCHEMA IF NOT EXISTS presentation;")

### 4.1 Presentation Layer - Latest Records for Subset

In [6]:
# Test query to see if we can create a ranked list
data = conn.query("""
    SELECT
        *,
        ROW_NUMBER() OVER (
            PARTITION BY location_id, sensors_id, "datetime", "parameter"
            ORDER BY ingestion_datetime DESC
        ) AS rn
    FROM raw.air_quality_data
    WHERE parameter IN ('pm25', 'o3')
    ORDER BY "datetime" ASC;
""").to_df()

data.head(5)

,location_id,sensors_id,location,datetime,lat,lon,parameter,units,value,month,year,ingestion_datetime,rn
0,384,671,CCNY-384,2025-01-01 06:00:00,40.8197,-73.9481,o3,ppm,0.024,01,2025,2025-08-17 10:18:31.944,1
1,384,673,CCNY-384,2025-01-01 06:00:00,40.8197,-73.9481,pm25,µg/m³,8.700,01,2025,2025-08-17 10:18:31.944,1
2,384,671,CCNY-384,2025-01-01 07:00:00,40.8197,-73.9481,o3,ppm,0.019,01,2025,2025-08-17 10:18:31.944,1
3,384,673,CCNY-384,2025-01-01 07:00:00,40.8197,-73.9481,pm25,µg/m³,11.600,01,2025,2025-08-17 10:18:31.944,1
4,384,671,CCNY-384,2025-01-01 08:00:00,40.8197,-73.9481,o3,ppm,0.009,01,2025,2025-08-17 10:18:31.944,1


In [7]:
conn.execute("""
    CREATE OR REPLACE VIEW presentation.air_quality_data AS (
        WITH ranked_data AS (
            SELECT
                *,
                ROW_NUMBER() OVER (
                    PARTITION BY location_id, sensors_id, "datetime", "parameter"
                    ORDER BY ingestion_datetime DESC
                ) AS rn
            FROM raw.air_quality_data
            WHERE parameter IN ('pm25', 'o3')
        )

        SELECT
            location_id, 
            sensors_id, 
            "location", 
            "datetime", 
            lat, 
            lon, 
            "parameter", 
            units, 
            "value",
            "month", 
            "year",
        FROM ranked_data
        WHERE rn = 1
    );
""")

In [8]:
data = conn.query("""
    SELECT *
    FROM presentation.air_quality_data;
""").to_df( )

data.head(5)

,location_id,sensors_id,location,datetime,lat,lon,parameter,units,value,month,year
0,384,671,CCNY-384,2025-01-03 02:00:00,40.8197,-73.9481,o3,ppm,0.018,01,2025
1,384,671,CCNY-384,2025-01-04 00:00:00,40.8197,-73.9481,o3,ppm,0.015,01,2025
2,384,671,CCNY-384,2025-01-06 01:00:00,40.8197,-73.9481,o3,ppm,0.028,01,2025
3,384,671,CCNY-384,2025-01-06 03:00:00,40.8197,-73.9481,o3,ppm,0.024,01,2025
4,384,671,CCNY-384,2025-01-06 09:00:00,40.8197,-73.9481,o3,ppm,0.014,01,2025


### 4.2 Presentation - Getting Latest Param Values for Location

In [9]:
# Temp Query
data = conn.query("""
    SELECT
        location_id,
        "location",
        "datetime", 
        lat, 
        lon, 
        "parameter", 
        "value",
        ROW_NUMBER() OVER (
            PARTITION BY location_id, "parameter"
            ORDER BY "datetime" DESC
        ) AS rn
    FROM presentation.air_quality_data
    ORDER BY "datetime" DESC;
""").to_df()

data.head(5)

,location_id,location,datetime,lat,lon,parameter,value,rn
0,384,CCNY-384,2025-07-01 04:00:00,40.8197,-73.9481,o3,0.034,1
1,384,CCNY-384,2025-07-01 04:00:00,40.8197,-73.9481,pm25,10.300,1
2,384,CCNY-384,2025-07-01 03:00:00,40.8197,-73.9481,o3,0.034,2
3,384,CCNY-384,2025-07-01 03:00:00,40.8197,-73.9481,pm25,10.100,2
4,384,CCNY-384,2025-07-01 02:00:00,40.8197,-73.9481,o3,0.037,3


In [10]:
conn.execute("""
    CREATE OR REPLACE VIEW presentation.latest_params_per_location AS (
        WITH ranked_data AS (
            SELECT
                location_id,
                "location",
                "datetime", 
                lat, 
                lon, 
                "parameter", 
                "value",
                ROW_NUMBER() OVER (
                    PARTITION BY location_id, "parameter"
                    ORDER BY "datetime" DESC
                ) AS rn
            FROM presentation.air_quality_data
        )

        PIVOT (
            SELECT
                location_id,
                "location",
                lat, 
                lon,
                "datetime", 
                "parameter", 
                "value"
            FROM ranked_data
            WHERE rn = 1
        )

        ON parameter IN ('pm25', 'o3')
        USING FIRST("value")
    );
""")

In [11]:
data = conn.query("""
    SELECT *
    FROM presentation.latest_params_per_location;
""").to_df( )

data.head(5)

,location_id,location,lat,lon,datetime,pm25,o3
0,384,CCNY-384,40.8197,-73.9481,2025-07-01 04:00:00,10.3,0.034


### 4.3 Presentation - Daily Stats

In [12]:
data = conn.query("""
            SELECT
                location_id,
                "location",
                CAST("datetime" as DATE) as measurement_date, 
                lat, 
                lon, 
                "parameter",
                units,
                "value",
                dayofweek("datetime") AS weekday_number,
                dayname("datetime") AS weekday,
            CASE
                WHEN dayname("datetime") IN ('Saturday', 'Sunday')
                THEN 1
                ELSE 0
            END AS is_weekend
            FROM presentation.air_quality_data
""").to_df()

data.head(5)

,location_id,location,measurement_date,lat,lon,parameter,units,value,weekday_number,weekday,is_weekend
0,384,CCNY-384,2025-01-01,40.8197,-73.9481,o3,ppm,0.015,3,Wednesday,0
1,384,CCNY-384,2025-01-02,40.8197,-73.9481,o3,ppm,0.029,4,Thursday,0
2,384,CCNY-384,2025-01-02,40.8197,-73.9481,o3,ppm,0.017,4,Thursday,0
3,384,CCNY-384,2025-01-03,40.8197,-73.9481,o3,ppm,0.020,5,Friday,0
4,384,CCNY-384,2025-01-05,40.8197,-73.9481,o3,ppm,0.030,0,Sunday,1


In [13]:
conn.execute("""
    CREATE OR REPLACE VIEW presentation.daily_stats AS (
        WITH dated_data AS (
            SELECT
                location_id,
                "location",
                CAST("datetime" as DATE) as measurement_date, 
                lat, 
                lon, 
                "parameter",
                units,
                "value",
                dayofweek("datetime") AS weekday_number,
                dayname("datetime") AS weekday,
            CASE
                WHEN dayname("datetime") IN ('Saturday', 'Sunday')
                THEN 1
                ELSE 0
            END AS is_weekend
            FROM presentation.air_quality_data
        )

        SELECT
            location_id,
            "location",
            measurement_date,
            weekday_number,
            weekday,
            is_weekend,
            lat,
            lon,
            parameter,
            units,
            AVG(value) AS average_value
        FROM dated_data
        GROUP BY
            location_id,
            location,
            measurement_date,
            weekday_number,
            weekday,
            is_weekend,
            lat,
            lon,
            parameter,
            units
    );
""")

In [14]:
data = conn.query("""
    SELECT *
    FROM presentation.daily_stats
    ORDER BY measurement_date;
""").to_df( )

data.head(5)

,location_id,location,measurement_date,weekday_number,weekday,is_weekend,lat,lon,parameter,units,average_value
0,384,CCNY-384,2025-01-01,3,Wednesday,0,40.8197,-73.9481,pm25,µg/m³,7.911111
1,384,CCNY-384,2025-01-01,3,Wednesday,0,40.8197,-73.9481,o3,ppm,0.016500
2,384,CCNY-384,2025-01-02,4,Thursday,0,40.8197,-73.9481,o3,ppm,0.022042
3,384,CCNY-384,2025-01-02,4,Thursday,0,40.8197,-73.9481,pm25,µg/m³,2.325000
4,384,CCNY-384,2025-01-03,5,Friday,0,40.8197,-73.9481,pm25,µg/m³,4.700000


In [15]:
data.shape

(126, 11)

In [16]:
data.parameter.value_counts()

parameter
pm25    63
o3      63
Name: count, dtype: int64

### 4.4 End

In [17]:
tables = conn.query("SHOW ALL TABLES;")
tables

┌─────────────┬──────────────┬────────────────────────────┬───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┬────────────────────────────────────────────────────────────────────────────────────────────────────────────┬───────────┐
│  database   │    schema    │            name            │                                                       column_names                                                        │                                                column_types                                                │ temporary │
│   varchar   │   varchar    │          varchar           │                                                         varchar[]                                                         │                                                 varchar[]                                                  │  boolean  │
├─────────────┼──────────────┼────────────────────────────┼──────────────────────────

In [19]:
# Closing Statement
conn.close()